In [36]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
# from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.remote.webelement import WebElement


In [37]:
def convert_openinghours(openingHours: dict) -> dict:
    temp_openingHours = openingHours.copy()

    if(len(temp_openingHours) == 0):
        return temp_openingHours.copy()
    
    # cut substring represent special holiday
    # for example:
    # {'วันจันทร์(วันเฉลิมพระชนมพรรษา พระบาทสมเด็จพระปรเมนทรรามาธิบดีศรีสินทรมหาวชิราลงกรณ พระวชิรเกล้าเจ้าอยู่หัว (วันหยุดชดเชย))': '9:00–17:00'}
    # change its key to -> {'วันจันทร์': '9:00–17:00'}
    for key, val in temp_openingHours.copy().items():
        start_Idx_special_holiday = key.find('(')
        if(start_Idx_special_holiday != -1):
            # changing keys of dictionary
            new_key = key[:start_Idx_special_holiday]
            temp_openingHours[new_key] = temp_openingHours.pop(key)

    days_of_week = ['อาทิตย์', 'จันทร์', 'อังคาร', 'พุธ', 'พฤหัสบดี', 'ศุกร์', 'เสาร์']
    # in case of temp_openingHours = {"ทุกวัน": '10:30 - 21:00'}
    # convert it to dictionary with all days of week as a keys(same value)
    if(len(temp_openingHours) == 1 and list(temp_openingHours.keys())[0] == 'ทุกวัน'):
        temp_time = list(temp_openingHours.values())[0]
        del temp_openingHours['ทุกวัน']
        for cur_day_of_week in days_of_week:
            temp_openingHours[cur_day_of_week] = temp_time

    else:
        # if there is range between day of week --> convert it to two individual key with same value
        # for example: {'จันทร์ - พุธ': '10:00 - 20:30', 'อาทิตย์': '11:00 - 22:30'}
        # convert to -> {'จันทร์': '10:00 - 20:30', 'อังคาร': '10:00 - 20:30', 'พุธ': '10:00 - 20:30', 'อาทิตย์': '11:00 - 22:30'}
        for key, val in temp_openingHours.copy().items():
            cur_split_day_range = key.split(' - ')
            if(len(cur_split_day_range) == 1):
                continue
            # remove current key
            del temp_openingHours[key]
            # convert to two individual key with same value
            is_pass_endDay = False
            cur_start_day = cur_split_day_range[0]
            cur_end_day = cur_split_day_range[1]
            cur_Idx = days_of_week.index(cur_start_day)
            while(not is_pass_endDay):
                if(days_of_week[cur_Idx] == cur_end_day):
                    is_pass_endDay = True
                temp_openingHours[days_of_week[cur_Idx]] = val
                cur_Idx = (cur_Idx + 1) % len(days_of_week)

    # change openingHours to temp_openingHours
    return temp_openingHours.copy()

In [38]:
def scrape_img():
    pass

In [39]:
def scrape_location():
    pass

In [40]:
def scrape_restaurant(link_to_restaurant: str, restaurant: Restaurant) -> None:
    # navigate to the current restaurant page and continue scraping more data
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    restaurant_page_driver = webdriver.Chrome(options=options) 
    restaurant_page_driver.get(link_to_restaurant)

    # find description
    description = ""
    try:
        try:
            click_read_more =  restaurant_page_driver.find_element(By.XPATH, '//*[@id="contentRow"]/div[1]/div[5]/p/span')
            click_read_more.click()
            time.sleep(2)
        except Exception as e:
            pass

        description = restaurant_page_driver.find_element(By.XPATH, '//*[@id="contentRow"]/div[1]/div[5]/p').text

    except Exception as e:
        pass

    print("description -> ", description)

    # find phones
    phones = []
    try:
        possible_phone_elements = restaurant_page_driver.find_elements(By.CLASS_NAME, 'jrmgwi')
        for cur_element in possible_phone_elements:
            cur_text = cur_element.text
            if("เบอร์โทร : " in cur_text):
                phones = cur_text.split("เบอร์โทร : ")[-1].split(", ").copy()
                break
    except Exception as e:
        print("no phones ...")

    print("phone -> ", phones)

    # find websites
    all_website_dict = {}
    try:
        container_website_elements = restaurant_page_driver.find_element(By.CLASS_NAME, 'kKDiaN')
        all_website_elements = container_website_elements.find_elements(By.CLASS_NAME, 'cXFOMU')
        for cur_website in all_website_elements:
            cur_website_name = cur_website.text
            cur_website_link = cur_website.get_attribute('href')
            all_website_dict[cur_website_name] = cur_website_link

    except Exception as e:
        print("no website ...")

    print(all_website_dict)

    # find price range
    priceRange = ""
    try:
        possible_priceRange_elements = restaurant_page_driver.find_elements(By.CLASS_NAME, 'hpJBMe')
        for cur_element in possible_priceRange_elements:
            cur_text = cur_element.text
            if("บาท" in cur_text):
                priceRange = cur_text.replace('(', '').replace(')', '')
                break
    except Exception as e:
        print("no price range ...")        
    
    print(priceRange)

    # find facilities
    facilities = []
    try:
        container_list_facilities = restaurant_page_driver.find_element(By.XPATH, '//*[@id="contentRow"]/div[2]/div/div[1]/div[2]/div/ul')
        all_list_facilities = container_list_facilities.find_elements(By.TAG_NAME, 'li')

        for cur_list_element in all_list_facilities:
            all_span_elements = cur_list_element.find_elements(By.TAG_NAME, 'span')
            allowed_className = "buIyWl"
            # not_allowed_className = "McJoy"
            for cur_span in all_span_elements:
                cur_class =  cur_span.get_attribute('class')
                is_allowed_facility_span = (allowed_className in cur_class)

                # check if is facility with correct mark symbol -> if it is then continue scrape for facility
                if(is_allowed_facility_span):
                    cur_text = cur_list_element.find_element(By.CLASS_NAME, 'fFYUJu').text
                    try:
                        cur_sub_text = cur_list_element.find_element(By.CLASS_NAME, 'gFBGSr').text
                        cur_text = ('%s %s') % (cur_text, cur_sub_text)
                    except Exception as e:
                        pass
                    
                    facilities.append(cur_text)
                    break

        print("cur facilities -> ", facilities)

    except Exception as e:
        print("no facilities ...")

    
    # find openingHours
    openingHours = {}
    try:
        all_openingHours_element = restaurant_page_driver.find_elements(By.CLASS_NAME, 'gdNTro')
        for cur_openingHours_element in all_openingHours_element:
            cur_all_td_elements = cur_openingHours_element.find_elements(By.TAG_NAME, 'td')
            cur_day = cur_all_td_elements[0].text
            cur_time = cur_all_td_elements[1].text
            openingHours[cur_day] = cur_time

    except Exception as e:
        print("no opening hours ...")

    openingHours = convert_openinghours(openingHours)
    print("cur opening hours: ")
    print(openingHours)
 
    # scrape location

    # scrape image path

    restaurant_page_driver.close()

In [41]:
def scrape_restaurants_by_province(province: str, wongnai_regionId: str) -> pd.DataFrame:
    # prepare option to create new driver for some scraping task
    # for example: get to google map or next page without click on the same browser(wongnai might not allowed) 
    # options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    
    res_restaurant_df = pd.DataFrame()
    is_last_page = False
    cur_query_url = "https://www.wongnai.com/restaurants?categoryGroupId=9&regions=%s" % (wongnai_regionId)
    
    # create webdriver instance using option to mazimize current window
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    cnt_for_debug = 0
    while(not is_last_page):
        if(cnt_for_debug == 2):
            break
        cnt_for_debug += 1

        driver = webdriver.Chrome(options=options)
        driver.get(cur_query_url)
        
        # find first group of restaurant on the first page
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'dibyTT')))
        all_restaurants_card = driver.find_elements(By.CLASS_NAME, 'dibyTT')
        all_clickable_elements = driver.find_elements(By.CLASS_NAME, 'fsElrZ')

        # iterate to scrape each restaurant and set propeties of 'Restaurant' object
        for cur_restaurant_card, cur_clickable_element in zip(all_restaurants_card, all_clickable_elements):
            cur_restaurant = Restaurant()

            # find restaurant name
            cur_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'Dtkmv').text
            cur_sub_name = ''
            # cut substring if there is sub name
            try:
                cur_sub_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'dqdias').text
                print("sub name -> ", cur_sub_name)
                cur_Idx_sub_name = cur_name.rfind(cur_sub_name)
                cur_name = cur_name[:cur_Idx_sub_name]
                
            except Exception as e:
                pass

            print('name -> ', cur_name)
            cur_restaurant.set_name(cur_name)
            cur_restaurant.set_sub_name(cur_sub_name)

            # find restaurant tags
            cur_restaurant_tags = []
            cur_restaurant_tags_elements = cur_restaurant_card.find_elements(By.CLASS_NAME, 'okmRN')
            for cur_element in cur_restaurant_tags_elements:
                cur_restaurant_tags.append(cur_element.text)
            print("cur tags")
            print(cur_restaurant_tags)

            # set some properties of 'Restaurant' object
            cur_restaurant.set_name(cur_name)
            cur_restaurant.set_sub_name(cur_sub_name)
            cur_restaurant.set_restaurantTag(cur_restaurant_tags)
        
            time.sleep(2)
            # navigate to the current restaurant page and continue scraping more data
            # get url via "href" attribute of tag 'a' -> navigate restaurant page
            link_to_restaurant = cur_clickable_element.get_attribute('href')
            scrape_restaurant(
                link_to_restaurant = link_to_restaurant,
                restaurant = cur_restaurant,
            )
                

            
        # find button to click next -> get url via "href" attribute of tag 'a' -> navigate to the next page in a new browser window (aviod wongnai might not allowed web scraping).
        time.sleep(2)
        next_page_element = driver.find_element(By.XPATH, '//*[@id="contentContainer"]/div[1]/div[2]/div[2]/div[1]/div[2]/a[2]')
        link_to_nextPage = next_page_element.get_attribute('href')
        # after scrap everything in current page -> check if it is the lastpage
        if(not link_to_nextPage):
            break

        # if not last page -> set new query to navigate to the next page in a new browser window(next iteration)
        cur_query_url = link_to_nextPage
        # close current browser tab    
        driver.close() 

    return res_restaurant_df.copy()


In [42]:
cnt_for_debug = 0
for cur_region_data in ALL_REGION_ID_WONGNAI[::-1]:
    if(cnt_for_debug == 1):
        break
    
    cur_province = cur_region_data[1]
    cur_regionId = cur_region_data[2]

    # get dataframe result of all restaurants in current province
    cur_res_allRestaurants_df = scrape_restaurants_by_province(
        wongnai_regionId = cur_regionId,
        province = cur_province
    )
    cnt_for_debug += 1
    # save result dataframe to .csv
    

name ->  Coffee Bar Ranong
cur tags
['คาเฟ่', 'อาหารไทย']
description ->  
phone ->  ['0839684963']
{'Instagram': 'https://instagram.com/coffee_bar_ranong'}
ต่ำกว่า 100 บาท
cur facilities ->  ['ที่จอดรถ (จอดข้างทาง)', 'Wi-Fi มี Wi-Fi บริการฟรี', 'เดลิเวอรี', 'เหมาะสำหรับเด็กๆ', 'เหมาะสำหรับมาเป็นกลุ่ม', 'แอลกอฮอล์ (เหล้า เบียร์ คอกเทล)']
cur opening hours: 
{'อาทิตย์': '09:30 - 19:00', 'จันทร์': '09:30 - 19:00', 'อังคาร': '09:30 - 19:00', 'พุธ': '09:30 - 19:00', 'พฤหัสบดี': '09:30 - 19:00', 'ศุกร์': '09:30 - 19:00', 'เสาร์': '09:30 - 19:00'}
sub name ->  เปิดทุกวัน 14.00-22.00
name ->  สนามบอลมิงเกิ้ล
cur tags
['คาเฟ่']
description ->  
phone ->  ['0933966838']
{'Facebook': 'https://www.wongnai.com/www.facebook/Milkcocafe', 'Instagram': 'https://instagram.com/@milkco.cafe', 'LINE@': 'http://line.naver.jp/ti/p/774swiju'}
101 - 250 บาท
cur facilities ->  ['ที่จอดรถ (มีที่จอดรถ)', 'Wi-Fi มี Wi-Fi บริการฟรี', 'เดลิเวอรี', 'เหมาะสำหรับมาเป็นกลุ่ม']
cur opening hours: 
{'อาทิตย์': '09:00 - 1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x00007FF600379632+30946]
	(No symbol) [0x00007FF60032E3C9]
	(No symbol) [0x00007FF600226FDA]
	(No symbol) [0x00007FF6001FCB85]
	(No symbol) [0x00007FF6002A37A7]
	(No symbol) [0x00007FF6002BA771]
	(No symbol) [0x00007FF60029C813]
	(No symbol) [0x00007FF60026A6E5]
	(No symbol) [0x00007FF60026B021]
	GetHandleVerifier [0x00007FF6004AF83D+1301229]
	GetHandleVerifier [0x00007FF6004BBDB7+1351783]
	GetHandleVerifier [0x00007FF6004B2A03+1313971]
	GetHandleVerifier [0x00007FF6003ADD06+245686]
	(No symbol) [0x00007FF60033758F]
	(No symbol) [0x00007FF600333804]
	(No symbol) [0x00007FF600333992]
	(No symbol) [0x00007FF60032A3EF]
	BaseThreadInitThunk [0x00007FFFA700257D+29]
	RtlUserThreadStart [0x00007FFFA7B0AF28+40]
